Для разбиения выборки в Python есть специальная функция test_train_split из библиотеки Scikit-learn:

In [6]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline
from sklearn.model_selection import train_test_split # функция, чтобы разбить данные на трейн и тест
from sklearn.linear_model import LogisticRegression # наша модель для классификации

In [7]:
from sklearn.model_selection import train_test_split

После этого мы должны обозначить нашу зависимую переменную (Y) и независимые (X) и с помощью этой функции создать обучающую и тестовую выборки:

In [8]:
X_train, X_test, y_train, y_test = model_selection.train_test_split(X, y, train_size=0.65,test_size=0.35, random_state=101)

NameError: name 'model_selection' is not defined

In [9]:
from sklearn.model_selection import KFold 
X = np.array([[1, 2], [3, 4], [1, 2], [3, 4]]) 
y = np.array([1, 2, 3, 4]) 
kf = KFold(n_splits=2)  #реализация разбиения
kf.get_n_splits(X) #возвращает количество разбиений
kf.split(X) #возвращает индексы для разбиения

<generator object _BaseKFold.split at 0x000001DC360179C8>

Посмотрим на работу hold-out — отложенной выборки на примере датасета с ирисами. Целевая переменная — класс цветка. Загрузим данные с помощью библиотеки Pandas:



In [11]:
import pandas as pd

iris_data = pd.read_csv('./data/iris.data', 
                        names=['sepal_length', 'sepal_width', 
                               'petal_length', 'petal_width', 'class'])


FileNotFoundError: [Errno 2] File ./data/iris.data does not exist: './data/iris.data'

У нас есть четыре признака, относящиеся к размерам цветка, и классы. На первом этапе мы разобьём выборку на две части: тренировочную и валидационную, а позже рассмотрим, как добавить тестовую. Воспользуемся методом train_test_split() из библиотеки sklearn:



In [12]:
from sklearn.model_selection import train_test_split

train, valid = train_test_split(iris_data, test_size=0.15, shuffle=True)

print('Размер исходной выборки: {}\nРазмер тренировочной выборки: {}\nРазмер валидационной выборки: {}'
      .format(iris_data.shape, train.shape, valid.shape)
)


NameError: name 'iris_data' is not defined

Теперь визуализируем наше разбиение с помощью специального метода:



In [14]:
def visualize_train_valid_counts(init_data, train, valid):
    x = np.array([0, 1, 2])
    width = 0.2

    plt.figure(figsize=(15, 8))
    ax = plt.subplot(111)

    classes = list(init_data['class'].value_counts().index)

    ax.bar(x - width, list(init_data['class'].value_counts()[classes]), width, color='r', label='Исходные данные')
    ax.bar(x, list(train['class'].value_counts()[classes]), width, color='g', label='Тренировочная выборка')
    ax.bar(x, list(valid['class'].value_counts()[classes]), width, bottom=list(train['class'].value_counts()[classes]), color='b', label='Валидационная выборка')

    ax.set_ylim([0, 70])
    plt.xticks(x - width / 2, classes, fontsize=20)
    plt.yticks(fontsize=15)
    plt.ylabel('Кол-во примеров', fontsize=20)
    plt.minorticks_on()
    plt.grid(which='major', color='r')
    plt.grid(which='minor', linestyle=':', color='k')
    plt.legend(fontsize=15)


У нас три класса цветков, в каждом из которых получилось разное количество примеров с выборками, в третьем примере значительно больше примеров на валидационной выборке. Получилось неравномерное распределение классов после разбиения, так как мы не учли доли классов.

Проблема решается с помощью stratified разбиения или стратифицированного разбиения. В библиотеке sklearn есть нужный нам класс StratifiedShuffleSplit:



In [15]:
from sklearn.model_selection import StratifiedShuffleSplit

sss = StratifiedShuffleSplit(n_splits=1, test_size=0.15)
train_indices, valid_indices = [split for split in sss.split(iris_data.iloc[:, :4], iris_data.iloc[:, 4])][0]
s_train = iris_data.iloc[train_indices]
s_valid = iris_data.iloc[valid_indices]
В класс мы передали количество разбиений, размер выборки и вызвали метод split() для более равномерного распределения. В результате разбиение получилось практически одинаковым.



SyntaxError: invalid syntax (<ipython-input-15-29b6d89bc427>, line 7)

Как разбить выборку на три части? Определить размер выборок и поделить дважды: сначала на тренировочную и остальное, затем остальное на валидационную и тестовую:



In [16]:
valid_frac = 0.15
test_frac = 0.15

valid_count = int(valid_frac * iris_data.shape[0])
test_count = int(test_frac * iris_data.shape[0])

train, valid_test = train_test_split(iris_data, 
                                     test_size=(valid_count + test_count))
valid, test = train_test_split(valid_test, test_size=test_count)


NameError: name 'iris_data' is not defined

Второй тип разбиения — k-fold, перекрёстная валидация. Сделаем k-fold разбиение данных, используя класс KFold:



In [17]:
from sklearn.model_selection import KFold

# shuffle=False исключает предварительное перемешивание примеров
kf = KFold(n_splits=3, shuffle=False)

for train_index, valid_index in kf.split(iris_data):
    print('train indices: {}\nvalid indices: {}'.format(train_index, valid_index))
    print('='*20)


NameError: name 'iris_data' is not defined

Ещё один способ разбиения — leave-one-out, отложенный пример. Сделаем leave-one-out разбиение пяти примерах из исходных данных с помощью класса LeaveOneOut:



In [ ]:
from sklearn.model_selection import LeaveOneOut

loo = LeaveOneOut()

for train_index, valid_index in loo.split(iris_data[:5]):
    print('train indices: {}\nvalid indices: {}'.format(train_index, valid_index))
    print('='*20)

In [ ]:
from sklearn.model_selection import train_test_split
vis_data = pd.read_csv("train.csv", encoding = 'ISO-8859-1', low_memory = False)